# Chat With Anything - From PDFs Files to Image Documents: 
Author: Zoumana KEITA   
https://medium.com/@zoumanakeita

In [55]:
import warnings
warnings.filterwarnings('ignore')

### Install the requirements

In [56]:
%%bash

pip -q install langchain faiss-cpu unstructured
pip -q install openai tiktoken
pip -q install pytesseract pypdf


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


# Chat & Query your PDF files

## Detect Document Type

In [63]:

from filetype import guess

def detect_document_type(document_path):

    guess_file = guess(document_path)
    file_type = ""
    image_types = ['jpg', 'jpeg', 'png', 'gif']

    if(guess_file.extension.lower() == "pdf"):
        #print(guess_file.extension.lower())
        file_type = "pdf"

    elif(guess_file.extension.lower() in image_types):

        file_type = "image"

    else:
        file_type = "unkown"

    return file_type


In [69]:
guess("1148_Notas_2023.png").extension.lower() == "image"

False

In [66]:
research_paper_path = "Banco BPI RC 2023port.pdf"
article_information_path = "1148_Notas_2023.png"

print(f"Research Paper Type: {detect_document_type(research_paper_path)}")
print(f"Article Information Document Type: {detect_document_type(article_information_path)}")

Research Paper Type: pdf
Article Information Document Type: pdf


## Extract Documents Content

In [51]:
from langchain.document_loaders.image import UnstructuredImageLoader
from langchain.document_loaders import UnstructuredFileLoader

"""
YOU CAN UNCOMMENT THE CODE BELOW TO UNDERSTAND THE LOGIC OF THE FUNCTIONS
"""
"""

def extract_text_from_pdf(pdf_file):

    loader = UnstructuredFileLoader(pdf_file)
    documents = loader.load()
    pdf_pages_content = '\n'.join(doc.page_content for doc in documents)

    return pdf_pages_content
b
def extract_text_from_image(image_file):

    loader = UnstructuredImageLoader(image_file)
    documents = loader.load()

    image_content = '\n'.join(doc.page_content for doc in documents)

    return image_content


"""

def extract_file_content(file_path):

    file_type = detect_document_type(file_path)

    if(file_type == "pdf"):
        loader = UnstructuredFileLoader(file_path)

    elif(file_type == "image"):
        loader = UnstructuredImageLoader(file_path)

    documents = loader.load()
    documents_content = '\n'.join(doc.page_content for doc in documents)

    return documents_content

In [52]:
#research_paper_content = extract_text_from_pdf(research_paper_path)
#article_information_content = extract_text_from_image(article_information_path)


#research_paper_content = extract_file_content(research_paper_path)
article_information_content = extract_file_content(article_information_path)

In [53]:
nb_characters = 3000

#print(f"First {nb_characters} Characters of the Paper: \n{research_paper_content[:nb_characters]}...")
#print("---"*5)
print(f"First Characters of Article Information Document :\n {article_information_content}...")
#article_information_content


First Characters of Article Information Document :
 2.NOTAS INTEGRANTES DAS DEMONSTRACOES FINANCEIRAS

Notas explicativas integrantes das demonstrag6es financeiras (Valores expressos em euros, exceto quando indicado) 1. InformagGes gerais

A Crédito Agricola Vida - Companhia de Seguros, S.A. (CA Vida) foi constituida por escritura publica, celebrada em 30 de novembro de 1998, sob a forma juridica de Sociedade Anénima, com o objetivo de desenvolver autonomamente a atividade de Seguros do Rarno Vida em Portugal. No ultimo trimestre de 2006, iniciou a atividade de gestao de fundos coletivos de reforma. A Companhia encontra-se registada em Portugal, tendo a sua sede na Rua Castilho, 233 em Lisboa. O local principal dos negocios é na Rua de Campolide, n° 372, em Lisboa.

O capital social da CA Vida é detido em 98,18% pela Crédito Agricola Seguros e Pensdes, SGPS, sendo o restante detido por setenta e trés Caixas Agricolas que se encontram detalhadas na Note 32.

As demonstracdes financeiras

## Chat Implementation

### Create Chunks

In [27]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

In [28]:
research_paper_chunks = text_splitter.split_text(research_paper_content)
article_information_chunks = text_splitter.split_text(article_information_content)

print(f"# Chunks in Research Paper: {len(research_paper_chunks)}")
print(f"# Chunks in Article Document: {len(article_information_chunks)}")

NameError: name 'research_paper_content' is not defined

### Create Embeddings

In [51]:
from langchain.embeddings.openai import OpenAIEmbeddings
import os

os.environ["OPENAI_API_KEY"] = "<YOUR KEY>"

embeddings = OpenAIEmbeddings()

### Create Vector Index

In [52]:
from langchain.vectorstores import FAISS

def get_doc_search(text_splitter):

    return FAISS.from_texts(text_splitter, embeddings)

In [54]:
doc_search_paper = get_doc_search(research_paper_chunks)
print(doc_search_paper)

### Start chatting with your document

In [68]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(OpenAI(), chain_type = "map_rerank",
                      return_intermediate_steps=True)

def chat_with_file(file_path, query):

    file_content = extract_file_content(file_path)
    file_splitter = text_splitter.split_text(file_content)

    document_search = get_doc_search(file_splitter)
    documents = document_search.similarity_search(query)

    results = chain({
                        "input_documents":documents,
                        "question": query
                    },
                    return_only_outputs=True)
    results = results['intermediate_steps'][0]

    return results

##### Chat with the image file

In [70]:
query = "What is the document about"

results = chat_with_file(article_information_path, query)

answer = results["answer"]
confidence_score = results["score"]

print(f"Answer: {answer}\n\nConfidence Score: {confidence_score}")

/usr/local/lib/python3.9/site-packages/langchain/chains/llm.py:303: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Answer:  The document is a quick summary of Zoumana’s article on Medium related to Data Science, Machine Learning and Machine Learning Operations. It covers topics such as statistics, model evaluation metrics, SQL queries, NoSQL courses, data visualization, MLOps, and Natural Language Processing. 

Confidence Score: 100


##### Chat with the PDF file

In [71]:
query = "Why is the self-attention approach used in this document?"

results = chat_with_file(research_paper_path, query)

answer = results["answer"]
confidence_score = results["score"]

print(f"Answer: {answer}\n\nConfidence Score: {confidence_score}")

Created a chunk of size 1140, which is longer than the specified 1000
/usr/local/lib/python3.9/site-packages/langchain/chains/llm.py:303: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Answer:  Self-attention is used in this document to compute representations of its input and output without using sequence-aligned RNNs or convolution, making the Transformer the first transduction model relying entirely on self-attention.

Confidence Score: 100


# Congratulations!  

Made with ❤️ by Zoumana KEITA